# Trabalho Final de Estágio Docente
## Estagiário: Douglas Macedo Sgrott
## Aluno: Eduarda Cristina Rosa
## Data de entrega: 21/06/2021 (segunda-feira)
## O trabalho está organizado em partes:
 - ### **Dataset: Onde você irá limpar e pre processar o dataset. Atribua a versão final do dataset em um dataframe chamado df.**
 - Separação dos dados: Aqui os dados são normalizados e divididos em Treino/Validação. Não precisa modificar o código.
 - ### **Arquitetura da Rede Neural: Onde você vai definir a arquitetura da rede neural.**
 - ### **Parâmetros de otimização da Rede Neural: Onde você vai definir outros parâmetros da rede neural.**
 - Visualização dos resultados: Onde os resultados são obtidos
 - Exemplos: Servir como exemplo de análise, data cleaning e pré-processamento.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# importing stuff
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# a lot of stuff
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# Z-score / outliers stuff
from scipy import stats

# Rede Neural stuff
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import plot_model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset
#### Coloque aqui seu data cleaning e seu pre-processamento e atribua o dataset para um dataframe chamado **df**

**Anotações**

Dataframe: Estrutura de duas dimensões como uma tabela (linha, coluna). 

Os dados podem ser trabalhados também utilizando a estrutura Serie, para dados unidimensionais como uma coluna unica de uma tabela por exemplo. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df

## Categorização:
categorical = ['animal', 'city', 'furniture']
numerical = ['area', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']
ordinal = ['rooms', 'bathroom', 'parking spaces', 'floor']

# Tamanho do dataset:
print("{} linhas\n{} colunas/features.".format(df.shape[0], df.shape[1]))

#infos: 
#df.info()

# Versão bem simples de dataset para servir como exemplo e para conseguir rodar o notebook inteiro
# df.drop(columns='city', inplace=True)
# df.drop(columns='floor', inplace=True)
# df.drop(columns='animal', inplace=True)
# df.drop(columns='furniture', inplace=True)
# df.dropna(inplace=True)

# df = aqui vem o dataset processado, SEM ESCALONAMENTO/NORMALIZADOR

# tabela
# df.describe()

In [ ]:
df

In [ ]:
sns.heatmap(df.corr())

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()


In [ ]:
# Retorna o nro da dados faltantes por coluna, em seguida exibe os valores faltantes totais em cada uma das 13 colunas
missing_values_count = df.isnull().sum()
missing_values_count[0:13]

In [ ]:
# Retorna o total de valores faltantes que temos
total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()

# Porcentagem de dados que está faltando
percent_missing = (total_missing/total_cells) * 100
print(percent_missing, "percent of the data is missing")

**Analisando os dados faltantes:**

In [ ]:
df['animal'].value_counts()

In [ ]:
#animal - Criei uma nova opção e setei como não informado onde havia dados faltantes. 

df['animal'] = df['animal'].fillna('not informed')
df['animal'].value_counts()

In [ ]:
# furniture - Criei uma nova opção e setei como não informado onde havia dados faltantes. 

df['furniture'] = df['furniture'].fillna('not informed')
df['furniture'].value_counts()

In [ ]:
# city - Criei uma nova opção e setei como não informado onde havia dados faltantes. 

df['city'] = df['city'].fillna('not informed')
df['city'].value_counts()

#A principio eu tinha setado como nao informado porém conversando com colegas e pesquisnando, vi que não seria muito interessante esse tipo de tratativa
#então optei por deletar as linhas que não informavam a cidade.

In [ ]:
# uso de mascara para esconder os dados inves de dar um drop
df = df[df['city'] != 'not informed']

In [ ]:
df['city'].value_counts()

Removendo os dados com - e vazios em floor

In [ ]:
df['floor'].unique()

In [ ]:
df = df[df['floor'] != '-']
df['floor'].unique()

In [ ]:
df['floor'].replace('', np.nan, inplace=True)
df.dropna(subset=['floor'], inplace=True)

df['floor'].unique()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()

In [ ]:
df['rooms'].value_counts()

In [ ]:
#Substiuindo os valores vazios pelo dado mais coerente que é a existencia de pelo menos um quarto com certeza.
#.fillna substitui o argumento nos dados ausentes
df['rooms'] = df['rooms'].fillna('1.0')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()

In [ ]:
df['parking spaces'].value_counts()

In [ ]:
#Substiuindo os valores vazios pelo dado que aparece mais 
#.fillna substitui o argumento nos dados ausentes
df['parking spaces'] = df['parking spaces'].fillna('1.0')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()

In [ ]:
df['bathroom'].value_counts()

In [ ]:
#Substiuindo os valores vazios pelo dado mais coerente que é a existencia de pelo menos um banheiro com certeza.
#.fillna substitui o argumento nos dados ausentes
df['bathroom'] = df['bathroom'].fillna('1.0')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()

Os dados restantes serão imputados.

In [ ]:
df_regression = pd.concat([df['area'], df['hoa (R$)'], df['rent amount (R$)'], df['property tax (R$)'], df['fire insurance (R$)'], df['total (R$)']], axis=1)
df_regression.head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(df_regression.values)

regr_output = imp_mean.transform(df_regression.values)

In [ ]:
df['area'] = regr_output[:, 0]
df['hoa (R$)'] = regr_output[:, 1]
df['rent amount (R$)'] = regr_output[:, 2]
df['property tax (R$)'] = regr_output[:, 3]
df['fire insurance (R$)'] = regr_output[:, 4]
df['total (R$)'] = regr_output[:, 5]


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Lidando com os dados ausentes
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(10, 15))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()

In [ ]:
df.isna().sum()

**Detecção de outliers**

In [ ]:
selected_columns = numerical 
# + ordinal
df[selected_columns].head()

In [ ]:
fig, axes = plt.subplots(ncols=6, figsize=(15, 5))

for i,col in enumerate(selected_columns):
    axes[i].boxplot(df[col])
    axes[i].set_title(col)

plt.tight_layout()

In [ ]:
def graph_scatter():
    fig, ax = plt.subplots()

    ax.scatter(x=df['total (R$)'], y=df['area'])
    ax.set_ylabel("area")
    ax.set_xlabel("total (R$)")
    plt.show()
graph_scatter()

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['area'] < 1250
df = df[mask]
mask = df['total (R$)'] < 30000
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter()

In [ ]:
import plotly.express as px

data = df[['property tax (R$)', 'rent amount (R$)']].dropna()
x = data['property tax (R$)'].values
y = data['rent amount (R$)'].values

mask1 = np.abs(stats.zscore(x)) < 3
mask2 = np.abs(stats.zscore(y)) < 3
mask = np.logical_and(mask1, mask2)

fig = px.scatter(df, x="property tax (R$)", y="rent amount (R$)")

fig.add_shape(type="rect",
    x0=min(x[mask]), y0=min(y[mask]), x1=max(x[mask]), y1=max(y[mask]),
    line=dict(color="Green", width=2,),
    opacity=0.2,
    fillcolor="Green",
)

fig.show()

In [ ]:
from scipy import stats

columns = ['area', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']

fig, axes = plt.subplots(nrows=len(columns), ncols=2, figsize=(15,30))

for i, col in enumerate(columns):
    # Plotamos o histograma na esquerda
    axes[i][0].hist(df[col], bins=50)
    
    # Obtemos a mascara booleana de possíveis outliers (de acordo com o valor de Z)
    mask = np.abs(stats.zscore(df[col].dropna())) < 3
    dado_filtrado = df[col].dropna()[mask]
    
    # Plotamops o histograma na direita, já filtrado
    axes[i][1].hist(dado_filtrado)
    
    # Aproveitamos para plotar no esquerda, onde nós "cortamos" o histograma, para gerar o hist da direita
    ymax = axes[i][0].get_yticks()[-1]
    axes[i][0].vlines(x=max(dado_filtrado), ymin=0, ymax=ymax, color="red")
    
    # Damos nomes aos gráficos
    axes[i][0].set_title("{} - sem filtro de outliers".format(col))
    axes[i][1].set_title("{} - com filtro de outliers".format(col))

In [ ]:
def graph_scatter():
    fig, ax1 = plt.subplots()

    ax1.scatter(x=df['hoa (R$)'], y=df['property tax (R$)'])
    ax1.set_ylabel("property tax (R$)")
    ax1.set_xlabel("hoa (R$)")
    plt.show()
graph_scatter()

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))
# df1 = df
mask = df['property tax (R$)'] < 6000
df = df[mask]
mask = df['hoa (R$)'] < 10000
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter()

In [ ]:
def graph_scatter():
    fig, ax1 = plt.subplots()

    ax1.scatter(x=df['rent amount (R$)'], y=df['fire insurance (R$)'])
    ax1.set_ylabel("fire insurance (R$)")
    ax1.set_xlabel("rent amount (R$)")
    plt.show()
graph_scatter()

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))
# df1 = df
mask = df['fire insurance (R$)'] < 250
df = df[mask]
mask = df['rent amount (R$)'] < 15100
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter()

In [ ]:
df

**Feature enginnering**

In [ ]:
df1 = df

df1['totalPerArea'] = df1['total (R$)'] / df1['area']

df1[['totalPerArea', 'total (R$)', 'area']]

In [ ]:
df2 = df
df2['firePerRent'] = df2['fire insurance (R$)']/df['rent amount (R$)']
df2[['firePerRent', 'fire insurance (R$)', 'rent amount (R$)']]

**Feature encoding**

In [ ]:
df['animal'].replace(to_replace='acept', value=1, inplace=True)
df['animal'].replace(to_replace='not acept', value=0, inplace=True)
df['animal'].replace(to_replace='not informed', value=2, inplace=True)

df['furniture'].replace(to_replace='furnished', value=1, inplace=True)
df['furniture'].replace(to_replace='not furnished', value=0, inplace=True)
df['furniture'].replace(to_replace='not informed', value=2, inplace=True)

df['city'].replace(to_replace='São Paulo', value=1, inplace=True)
df['city'].replace(to_replace='Rio de Janeiro', value=2, inplace=True)
df['city'].replace(to_replace='Belo Horizonte', value=3, inplace=True)
df['city'].replace(to_replace='Porto Alegre', value=4, inplace=True)
df['city'].replace(to_replace='Campinas', value=5, inplace=True)

In [ ]:
# # OneHot Encoding
# df = df[df['city'].notna()]
# enc = OneHotEncoder(handle_unknown='ignore')
# enc.fit(df[['city']])
# enc_df = pd.DataFrame(enc.transform(df[['city']]).toarray(), columns=enc.get_feature_names(['city']))
# df.reset_index(drop=True, inplace=True)
# enc_df.reset_index(drop=True, inplace=True)
# df = df.join(enc_df)

df

# Separação dos dados

In [ ]:
# Normalizamos os dados de df em uma escala de [0, 1]
# Estou fazendo isto aqui pois temos que "desnormalizar" na hora de gerar os gráficos de R²
column_names = df.columns
scaler = MinMaxScaler()
scaler.fit(df)
df = scaler.transform(df)
df = pd.DataFrame(df)
df.columns = column_names

# Pegamos o dataset df e separamos em x (entrada) e y (saida), numa separação 70% treino e 30% validação
input_dim = df.shape[1] - 1
x = df.drop(columns='total (R$)')
y = df['total (R$)']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.30, random_state=42)

# Arquitetura da Rede Neural
#### Criei um código bem simples pra permitir criar diferentes redes neurais modificando apenas algumas variáveis (EM CAPSLOCK),
#### Mas se quiser criar sua própria arquitetura mais customizada, fique a vontade

In [ ]:
NEURONIOS_CAMADA_INICIAL = 10

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [20, 30]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

# Parâmetros de otimização da Rede Neural
#### Pode alterar os valores das variáveis que estão EM CAPSLOCK

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)


# Visualização dos resultados

In [ ]:

df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 2**

In [ ]:
NEURONIOS_CAMADA_INICIAL = 16

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)


In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 3**

In [ ]:
NEURONIOS_CAMADA_INICIAL = 16

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32, 16, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)


In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 4** - Função de ativação tanh

In [ ]:
NEURONIOS_CAMADA_INICIAL = 16

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32, 16, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'tanh'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)


In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 5**

In [ ]:
NEURONIOS_CAMADA_INICIAL = 32

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32, 16, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)

In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 6** - Usando regularizador

In [ ]:
NEURONIOS_CAMADA_INICIAL = 32

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32, 16, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = 'l1'

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)

In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

**Teste 7** - Usando dropout

In [ ]:
NEURONIOS_CAMADA_INICIAL = 32

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [64, 32, 16, 32]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = True

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'relu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

In [ ]:
CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)

In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()

# Lembrando:

## **Data de entrega:**
#### - 21/06/2021 (segunda-feira)

## **O que será avaliado de forma objetiva:**
#### - Se o modelo está ajustado (sem overfit/underfit). Ou seja, se as curvas de treino (azul) e validação (laranja) estão suficientemente próximos.
#### - A quantidade de dados (linhas de df) e features (colunas de df) foram utilizados.
#### - O valor de R²

## **O que será avaliado de forma subjetiva:**
#### - Esforço. Ou seja, quando mais explorar as combinações de pre-processamento + parâmetros de rede, melhor. Para isso, crie novas versões do notebook.

## Observações
#### - O valor de R será analisado juntamente com o número de features e dados usados. Ou seja, um R alto obtido usando poucos dados e features não vale muita coisa. Mas é melhor que nada ;)
#### - Se deixou de utilizar colunas do df = menos pontos. Se fez engenharia de features = mais pontos (pouco). Isto serve para incentivar a utilizar os dados categóricos.

## Dicas:
#### - Quando alcançar um resultado que queira salvar, clique em Save Version -> Quick Save. Isto fará com que o seus resultados atuais sejam mostrados na versão html. Se clicar em Save Version -> Save & Run All (commit), o notebook será rodado novamente, e isso pode modificar os resultados obtidos.
#### - No começo, KISS (Keep it simple, silly). Comece de forma simples. Comece com menos features e com pouco pre processamento. Recomendo começar usando variáveis numéricas e filtro de outliers. Isso já deve ser suficiente para obter um R alto. MAS ISSO NÃO É SUFICIENTE! Depois que conseguir um modelo com alto R usando um modelo e pre processamento simples, salve a versão do notebook e continue explorando! A meta é conseguir utilizar TODAS as features e o maior número de dados possível. O objetivo é explorar a combinação de pré-processamento de dados + parâmetros da rede neural.
#### - Eu vou analisar as diferentes versões do notebook (se houver). Ou seja, se na primeira versão obteve um modelo bom usando poucas features, ok... Se na segunda versão obteve um modelo um pouco pior ou melhor usando mais features, melhor ainda. Explore as combinações! You're a wizard Harry!

# FIM DO TRABALHO /\
# **INICIO DOS EXEMPLOS V**


# Dataset Original

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df

# Análise de correlação entre as variáveis
#### Pode ser interessante na hora de imputação

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

sns.heatmap(df.corr())

# Análise de caractéres especiais

In [ ]:
df.dtypes

#### É esperado que 'city', 'animal' e 'furniture'sejam considerados como tipo 'object', pois nestas colunas existem valores do tipo string. No entanto, 'floor' deveria ser considerado float64. Por que 'floor' está sendo considerado como object?

In [ ]:
df['floor'].unique()

#### Olhando os valores presentes em 'floor', percebemos que existe o caractér '-' presente. Além disso, não existem valores iguais a 0. Temos algumas possibilidades aqui:
 - Considerar '-' como 0 e fazer esta substituição (onde '-' significaria andar 0 ou térreo)
 - Deletar todas as linhas com floor igual a '-'

#### Vamos aproveitar também e verificar os valores de 'city', 'animal' e 'furniture'

In [ ]:
columns = ['city', 'animal', 'furniture']

for col in columns:
    print(df[col].unique())

# Análise de Outliers

### Vamos usar o PairPlot do Seaborn para traçar o conjunto de histogramas e scatterplots das variáveis contínuas:
#### 'hoa', 'rent amount', 'property tax', 'fire insurance', 'total'

In [ ]:
# Comentei isso aqui pq demora batante pra gerar a imagem. Se quiser ver, pode descomentar.
# sns.pairplot(df[['hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']])

#### É perceptível que existem outliers. Vamos traçar um Scatter Plot de 2 destas variáveis usando Plotly para criar um gráfico interativo para analisar um pouco melhor alguns desses outliers. Vamos também aplicar o Z-Score com z=3 para ter uma noção da faixa dos valores dos outliers.

In [ ]:
import plotly.express as px

data = df[['property tax (R$)', 'rent amount (R$)']].dropna()
x = data['property tax (R$)'].values
y = data['rent amount (R$)'].values

mask1 = np.abs(stats.zscore(x)) < 3
mask2 = np.abs(stats.zscore(y)) < 3
mask = np.logical_and(mask1, mask2)

fig = px.scatter(df, x="property tax (R$)", y="rent amount (R$)")

fig.add_shape(type="rect",
    x0=min(x[mask]), y0=min(y[mask]), x1=max(x[mask]), y1=max(y[mask]),
    line=dict(color="Green", width=2,),
    opacity=0.2,
    fillcolor="Green",
)

fig.show()

#### Vamos plotar os Histogramas dessas variáveis contínuas sem filtros e com filtros de outliers (Z-Score com z=3)

In [ ]:
columns = ['area', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']

fig, axes = plt.subplots(nrows=len(columns), ncols=2, figsize=(15,30))

for i, col in enumerate(columns):
    # Plotamos o histograma na esquerda
    axes[i][0].hist(df[col], bins=50)
    
    # Obtemos a mascara booleana de possíveis outliers (de acordo com o valor de Z)
    mask = np.abs(stats.zscore(df[col].dropna())) < 3
    dado_filtrado = df[col].dropna()[mask]
    
    # Plotamops o histograma na direita, já filtrado
    axes[i][1].hist(dado_filtrado)
    
    # Aproveitamos para plotar no esquerda, onde nós "cortamos" o histograma, para gerar o hist da direita
    ymax = axes[i][0].get_yticks()[-1]
    axes[i][0].vlines(x=max(dado_filtrado), ymin=0, ymax=ymax, color="red")
    
    # Danos nomes aos gráficos
    axes[i][0].set_title("{} - sem filtro de outliers".format(col))
    axes[i][1].set_title("{} - com filtro de outliers".format(col))

#### É possível remover estas variáveis outliers conforme indica o gráfico. Além disso, o valor de Z é ajustável: consideramos z=3, mas caso queira mais rigor, pode diminuir o valor de z ou aumentá-lo, caso queira um maior relaxamento na remoção de outliers.

# Distribuição das classes e dos números discretos

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

columns = ['city', 'animal', 'furniture']

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

df['city'].value_counts().plot.barh(ax=axes[0])
df['animal'].value_counts().plot.barh(ax=axes[1])
df['furniture'].value_counts().plot.barh(ax=axes[2])

axes[0].set_title("city")
axes[1].set_title("animal")
axes[2].set_title("furniture")
plt.tight_layout()


In [ ]:
# Para os numeros discretos:
columns = ['rooms', 'bathroom', 'parking spaces', 'floor']

df_aux = df.copy()
df_aux['floor'].replace(to_replace='-', value=0, inplace=True)
df_aux['floor'] = pd.to_numeric(df_aux['floor'].values)

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 5))

for i, col in enumerate(columns):
    axes[i].hist(df_aux[col], bins=df_aux[col].nunique())
    axes[i].set_title(col)

#### Como estamos falando de número de vagas de estacionamentos, quartos e banheiros, o esperado é que a maior parte dos dados esteja nos valores pequenos, formando uma distribuição assimétrica. Ainda assim, existem alguns casos... atípicos, como por exemplo um imóvel com 12 quartos. Isso é um outlier? Depende! Se for um hotel, 12 quartos pode ser considerado pouco.

# Variáveis ausentes

#### Vamos plotar o número de dados ausentes por coluna em um Gráfico de Barras

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

df.isna().sum().plot.barh()
plt.show()

#### Cerca de 10% dos dados de cada COLUNA estão ausentes.
#### Vamos verificar a ausência de dados por LINHA:

In [ ]:
df.isna().sum(axis=1).value_counts().plot.barh()

#### A maioria das LINHAS possuem 1 dado ausente. Boa parte (cerca de 2900, segundo o gráfico) não contém NENHUM dado ausente. Outra boa parte dos dados possuem 2 dados ausentes.

#### Com base nisso, existem algumas perguntas:
 - Quais variáveis valem a pena imputar os dados ausentes?
 - Quais variáveis vale a pena deletar os dados ausentes?
 - Se formos deletar linhas com variáveis ausentes, qual seria nossa estratégia? Deletamos linhas com mais de 3 var ausentes ou todas?
 
#### Existem várias respostas, e todas elas tem prós e contras e dependendem do dataset e do contexto de modelagem.

# Exemplo de imputação dos valores ausentes das variáveis contínuas:
#### 'hoa', 'rent amount', 'property tax', 'fire insurance', 'total'

#### Primeiro vamos tratar os dados para criar um regressor com dados "limpos". Com base nesse regressor "limpo", vamos fazer uma imputação dos dados ausentes do dataframe. Vamos também comparar o resultado obtido com o resultado da imputação usando um regressor "sujo", gerado a partir de dados não tratados.

### Imputação com dados tratados:

In [ ]:
df_original = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df = df_original.copy()
# Faremos agora algumas imputações de valores ausentes usando regressão
# Para CRIAR o IMPUTADOR, vamos utilizar dados LIMPOS. Ou seja, sem dados ausentes e sem outliers.
columns = ['rent amount (R$)', 'property tax (R$)', 'area', 'total (R$)']
df_regress = df_original[columns].copy()

for col in columns:
    value = df_original[col].median()
    df_regress[col].fillna(value=value, inplace=True)

mask = (np.abs(stats.zscore(df_regress)) < 3).all(axis=1)
df_regress = df_regress[mask]

df_regress.describe()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Agora criaremos o regressor com os dados limpos
# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_regress.values)

# Agora, iremos pegar o dataset "sujo" e imputar valores nulos com o regressor "limpo"
X = df_original[['rent amount (R$)', 'property tax (R$)', 'area', 'total (R$)']].values
regr_output = imp_mean.transform(X)

foo = pd.DataFrame(regr_output)
foo.columns = df_regress.columns

for col in foo.columns:
    valores_menor_0 = (foo[col]<0).sum()
    print("Existem {} dados com valor < 0 em '{}'".format(valores_menor_0, col))

df[columns] = foo[mask]
    
foo[mask].describe()

### Imputação com dados não tratados:

In [ ]:
df_original = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df = df_original.copy()
# Faremos agora algumas imputações de valores ausentes usando regressão

columns = ['rent amount (R$)', 'property tax (R$)', 'area', 'total (R$)']
df_regress = df_original[columns].copy()
df_regress.dropna(inplace=True)

df_regress.describe()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Agora criaremos o regressor com os dados limpos
# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_regress.values)

# Agora, iremos pegar o dataset "sujo" e imputar valores nulos com o regressor "limpo"
X = df_original[['rent amount (R$)', 'property tax (R$)', 'area', 'total (R$)']].values
regr_output = imp_mean.transform(X)

foo = pd.DataFrame(regr_output)
foo.columns = df_regress.columns

for col in foo.columns:
    valores_menor_0 = (foo[col]<0).sum()
    print("Existem {} dados com valor < 0 em '{}'".format(valores_menor_0, col))

df[columns] = foo[mask]
    
foo[mask].describe()

# Exemplo de imputação dos valores ausentes das variáveis discretas:
#### 'parking spaces', 'rooms', 'bathroom'

#### Várias estratégias podem ser usadas. Aqui, usaremos uma regressão, mas iremos arredondar os números gerados para continuarem discretos.

#### Para esta imputação, usaremos 'rooms', 'bathroom' e 'parking spaces' que são valores discretos mas usaremos também 'rent amount' pois todas essas variáveis tem UM ALTO VALOR DE CORRELAÇÃO, e isso é importante na construção do regressor.

In [ ]:
df_original = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df = df_original.copy()

# Faremos agora algumas imputações de valores ausentes usando regressão
columns = ['rooms', 'bathroom', 'parking spaces', 'rent amount (R$)']
df_regress = df_original[columns].copy()

for col in columns:
    value = df_original[col].median()
    df_regress[col].fillna(value=value, inplace=True)

mask = (np.abs(stats.zscore(df_regress)) < 3).all(axis=1)
df_regress = df_regress[mask]

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_regress.values)

# Agora, iremos pegar o dataset "sujo" e imputar valores nulos com o regressor
X = df_original[columns].values
regr_output = np.round(imp_mean.transform(X))

foo = pd.DataFrame(regr_output)
foo.columns = df_regress.columns

for col in ['rooms', 'bathroom', 'parking spaces']:
    print("{}: {}".format(col, foo[col].unique()))
    

df[columns] = foo[mask]

foo[mask].describe()

# Exemplo de imputação dos valores ausentes de variáveis categóricas:
#### 'city', 'animal', 'furniture'

- Podemos substituir as variáveis ausentes pela moda dos valores (o número que mais se repete)
- Podemos substituir o dado ausente pelo dado anterior/seguinte
- Podemos criar uma nova categoria "Indefinido".

In [ ]:
df_antes = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)
df_depois = df_antes.copy()

# Substituindo dados ausentes pela categoria com maior frequência
df_depois['animal'].fillna(value=df_depois['animal'].mode()[0], inplace=True)

# Substituindo dados ausentes por "Indefinido"
df_depois['furniture'].fillna(value="Indefinido", inplace=True)

# Substituindo dados ausentes pelo próximo dado presente
df_depois.fillna(method='ffill', inplace=True)

fig, axes = plt.subplots(3, 1, figsize=(15, 10))

for i, col in enumerate(['city', 'animal', 'furniture']):
    axes[i].barh(width=df_depois[col].value_counts(), y=df_depois[col].unique(), label='depois')
    axes[i].barh(width=df_antes[col].value_counts(), y=df_antes[col].dropna().unique(), label='antes')
    axes[i].legend()
    
axes[0].set_title("Imputação copiando o dado anterior. Resultado proporcional", fontsize=18)
axes[1].set_title("Imputação usando a moda. Aumenta a desproporção", fontsize=18)
axes[2].set_title("Criação de uma nova classe", fontsize=18)
plt.tight_layout()
plt.show()

# Exemplo Feature Encoding

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

df['animal'].replace(to_replace='acept', value=1, inplace=True)
df['animal'].replace(to_replace='not acept', value=0, inplace=True)

df['furniture'].replace(to_replace='furnished', value=1, inplace=True)
df['furniture'].replace(to_replace='not furnished', value=0, inplace=True)

df

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

# OneHot Encoding
df = df[df['city'].notna()]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[['city']])
enc_df = pd.DataFrame(enc.transform(df[['city']]).toarray(), columns=enc.get_feature_names(['city']))
df.reset_index(drop=True, inplace=True)
enc_df.reset_index(drop=True, inplace=True)
df = df.join(enc_df)

df

# Exemplo de Feature Engineering (bem simples)
#### Criar novas features a partir de features já existentes
#### Obs: Não é obrigatório criar novas Features neste trabalho

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

df['Feature Nova 1'] = df['area'] ** 2
df['Feature Nova 2'] = df['fire insurance (R$)']/df['rent amount (R$)']
df['Feature Nova 3'] = df['rooms'] * df['bathroom']

df

# Feature Selection
#### Útil para decidir qual feature seria mais interessante de descartar. Estou inserindo apenas para fins de completude.

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

df.dropna(inplace=True)
df['floor'].replace(to_replace='-', value=0, inplace=True)
df['floor'] = pd.to_numeric(df['floor'].values)

columns = ['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)']
x = df[columns]
y = df['total (R$)']
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

# k é o numero de features que NÃO serão jogadas foras. Vamos primeiro ver os resultados, depois eliminar alguma feature.
k = x.shape[1]
# Utilizamos um método do sklearn para isso, usando a estratégia Chi Squared.
selector = SelectKBest(f_regression, k=k)
x_new = selector.fit_transform(x, y)

In [ ]:
# Utilizo o log10 pois os valores são ou muito grandes, ou muito pequenos
scores = -np.log10(selector.pvalues_)

x_plot = list(range(len(scores)))

fig, ax = plt.subplots(figsize=(8, 4))
plt.bar(x_plot, scores)
ax.set_title("Score do método f-regression para Feature Selection")
ax.set_xticks(x_plot)
ax.set_xticklabels(columns, rotation=45)
plt.show()